In [ ]:
# install the required packages

# !pip install pandas
# !pip install numpy
# !pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split

<span>Now we create first class, called Node, which contains some attributes. 
Most important from them are left and right, that represent child nodes</span>

In [2]:
class Node:
    """A node in a decision tree."""
    def __init__(self, X=None, y=None, feature=None,\
                  threshold=None, left=None, right=None, value=None):
        self.X = X
        self.y = y
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        # only leaf nodes have a value
        self.value = value

<span>Here's class of DecisionTreeClassifier
With method fit you create your tree
With method predict you logically predict classes of iris flowers 
And using method evaluate gives you oppurtunity to see how accurate this prediction is</span>

In [3]:
class MyDecisionTreeClassifier:
    """A decision tree classifier."""

    def __init__(self, max_depth):
        self.root = None
        self.max_depth = max_depth

    def info_gain(self, y, X, threshold, feature):
        '''
        Information gain is the difference in entropy before a split
        and the weighted average of the entropy after the split.
        '''
        parent_ent = self.entropy(y)

        left = y[X[feature] < threshold]
        right = y[X[feature] >= threshold]

        left_ent = self.entropy(left)
        right_ent = self.entropy(right)

        if len(left) == 0 or len(right) == 0:
            return 0

        n_l, n_r = len(left), len(right)
        n = len(y)
        child_ent = (n_l/n)* left_ent + (n_r/n)*right_ent

        return parent_ent - child_ent


    def entropy(self, classes):
        '''
        Entropy is a measure of the impurity of a set of examples.
        The higher the entropy, the more mixed the classes are.
        '''
        # calculate the frequency of each class
        hist = np.bincount(classes)
        ps = hist / len(classes)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def split_data(self, X, y) -> tuple[int, float, float]:
        """Find the best feature and threshold to split the data on."""

        best_split = (0, 0)
        features = X.columns
        gain = -1
        for feature in features:
            thresholds = np.unique(X[feature])
            for threshold in thresholds:

                info_gain = self.info_gain(y, X, threshold, feature)

                # if the information gain is greater than the current gain
                # update the best split
                if info_gain > gain:
                    best_split = (feature, threshold)
                    gain = info_gain
        # return gain as well so I don't have to calculate it again in build_tree
        return best_split[0], best_split[1], gain


    def build_tree(self, X, y, depth):
        """Build a decision tree."""

        if depth <= self.max_depth:
            best_split = self.split_data(X, y)
            feature, threshold = best_split[:2]
            # check whether the ig is greater than 0
            if best_split[-1] > 0:
                left_subtree = self.build_tree(X[X[feature] < threshold],\
                                                y[X[feature] < threshold], depth + 1)
                right_subtree = self.build_tree(X[X[feature] >= threshold],\
                                                 y[X[feature] >= threshold], depth + 1)
                return Node(X, y, feature, threshold, left_subtree, right_subtree)

        # if the depth is greater than the max depth
        # or the information gain is less than 0
        # return a leaf node with value of frequency of the most common class
        leaf_value = np.argmax(np.bincount(y))
        return Node(value=leaf_value)

    def fit(self, X, y):
        """Fit the decision tree to the data."""
        X = pd.DataFrame(X)
        self.root = self.build_tree(X, y, 0)

    def predict(self, X_test):
        """Make predictions using the decision tree."""

        def make_prediction(x, node):
            """Recursively traverse the tree to make predictions."""
            # if value is not None, then it's a leaf node
            # so we can return the value
            if node.value is not None:
                return node.value

            if x[node.feature] < node.threshold:
                return make_prediction(x, node.left)
            return make_prediction(x, node.right)

        return np.array([make_prediction(x, self.root) for x in X_test])

    def evaluate(self, X_test, y_test):
        """Evaluate the decision tree's accuracy on the test set."""

        y_pred = self.predict(X_test)
        return np.sum(y_pred == y_test) / len(y_test)

In [30]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [36]:
my_tree = MyDecisionTreeClassifier(max_depth=1)
my_tree.fit(X, y)
preds = my_tree.predict(X_test)
my_tree.evaluate(X_test, y_test)

0.9210526315789473